In [1]:
import mysql.connector
import json
import os
from datetime import datetime

In [2]:
# connecting to MySQL database
db = mysql.connector.connect(
    host='localhost',
    user='root',
    password='root1234',
    database='spotify_rec'
)

cursor = db.cursor()

In [3]:
# Function to insert playlist data into the Playlists table 
def insert_playlist(cursor, playlist):
    # Convert Unix timestamp to datetime
    modified_at_datetime = datetime.utcfromtimestamp(int(playlist["modified_at"]))

    sql = "INSERT IGNORE INTO Playlists (name, collaborative, pid, modified_at, num_tracks, num_albums, num_followers) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, (
        playlist["name"],
        1 if playlist["collaborative"].lower() == "true" else 0,
        playlist["pid"],
        modified_at_datetime.strftime('%Y-%m-%d %H:%M:%S'),
        playlist["num_tracks"],
        playlist["num_albums"],
        playlist["num_followers"]
    ))
    return cursor.lastrowid

# Function to insert track data into the Tracks table
def insert_track(cursor, track, playlist_pid):
    sql = "INSERT IGNORE INTO Tracks (pos, track_name, artist_name, track_uri, artist_uri, album_uri, duration_ms, album_name, pid) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, (
        track["pos"],
        track["track_name"],
        track["artist_name"],
        track["track_uri"],
        track["artist_uri"],
        track["album_uri"],
        track["duration_ms"],
        track["album_name"],
        playlist_pid
    ))
    return cursor.lastrowid


In [4]:
data_folder = "/Users/kashishmehta/Library/CloudStorage/OneDrive-RutgersUniversity/spotify_million_playlist_dataset/data"


In [ ]:
# Create a list of JSON file names
json_files = [filename for filename in os.listdir(data_folder) if filename.endswith(".json")]

# Sort JSON files in order
json_files.sort()

# Limit to the first 200 files
json_files = json_files[:200]

# Iterate through the selected JSON files
for json_file in json_files:
    
    json_file_path = os.path.join(data_folder, json_file)

    # Load and process each JSON file
    with open(json_file_path) as json_file:
        data = json.load(json_file)

        for playlist in data["playlists"]:
            # Insert playlist data into the Playlists table 
            playlist_pid = insert_playlist(cursor, playlist)

            for track in playlist["tracks"]:
                # Insert track data into the Tracks table
                insert_track(cursor, track, playlist_pid)



In [ ]:
# close the database connection
db.commit()
cursor.close()
db.close()